<a href="https://colab.research.google.com/github/Smart-Pigs/Depth-Images/blob/main/Weight_Estimation_ResNet34_Cropped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure you have this folder as shortcut in your drive - /content/gdrive/MyDrive/Morota-video-data/ 

In [ ]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


### Data Loading

In [ ]:
#@markdown Select resolution of input dataset and weeks of data

Instantiate_New_Dataset = True #@param {type:"boolean"}
Refresh_Old_Dataset = False #@param {type:"boolean"}

if Instantiate_New_Dataset == True:
  !rm -rf Week*/

Resolution = "256"  #@param ['720', '512', '256']

if Refresh_Old_Dataset == False:
  if Resolution == "720":
    !cp /content/gdrive/MyDrive/SmartPig/References/Morota-video-data/DEPTH_INDIVIDUAL/CROPPED/CROPPED_WEEK1-10_720.zip .
    !unzip -qq CROPPED_WEEK1-10_720.zip -d .
    !rm -rf *WEEK*.zip

  if Resolution == "512":
    !cp /content/gdrive/MyDrive/SmartPig/References/Morota-video-data/DEPTH_INDIVIDUAL/CROPPED/CROPPED_WEEK1-10_512.zip .
    !unzip -qq CROPPED_WEEK1-10_512.zip -d .
    !rm -rf *WEEK*.zip

  if Resolution == "256":
    !cp /content/gdrive/MyDrive/SmartPig/References/Morota-video-data/DEPTH_INDIVIDUAL/CROPPED/CROPPED_WEEK1-10_256.zip .
    !unzip -qq CROPPED_WEEK1-10_256.zip -d .
    !rm -rf *WEEK*.zip

### Mapping Table Input

In [ ]:
import pandas as pd

In [ ]:
final_map = pd.read_csv("/content/gdrive/MyDrive/SmartPig/References/Morota-video-data/final_mapping.csv")

In [ ]:
weeks = final_map

In [ ]:
weeks

,image_path,Date,Pig,weight,Week
0,Week1/20210922/20210922_3330_b/_Depth_4683.jpg,20210922,3330,15.5,1
1,Week1/20210922/20210922_3330_b/_Depth_4778.jpg,20210922,3330,15.5,1
2,Week1/20210922/20210922_3330_b/_Depth_4579.jpg,20210922,3330,15.5,1
3,Week1/20210922/20210922_3330_b/_Depth_4578.jpg,20210922,3330,15.5,1
4,Week1/20210922/20210922_3330_b/_Depth_4584.jpg,20210922,3330,15.5,1
...,...,...,...,...,...
96220,Week10/20211119/20211119_3342/_Depth_3504.jpg,20211119,3342,56.6,10
96221,Week10/20211119/20211119_3342/_Depth_3318.jpg,20211119,3342,56.6,10
96222,Week10/20211119/20211119_3342/_Depth_3476.jpg,20211119,3342,56.6,10
96223,Week10/20211119/20211119_3342/_Depth_3169.jpg,20211119,3342,56.6,10


In [ ]:
weeks_train = weeks[weeks["Pig"] != 3384]
# weeks_test = weeks[weeks['image_path'].str.contains('Week9|Week10')]

In [ ]:
weeks_test = weeks[weeks["Pig"] == 3384]
# weeks_train = weeks[~weeks['image_path'].str.contains('Week9|Week10')]

In [ ]:
image_paths_train = weeks_train['image_path'].values
labels_train = weeks_train['weight'].values
image_paths_test = weeks_test['image_path'].values
labels_test = weeks_test['weight'].values

### Dataset

In [ ]:
import tensorflow as tf
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((image_paths_train, labels_train))
test_dataset = tf.data.Dataset.from_tensor_slices((image_paths_test, labels_test))

In [ ]:
IMG_SIZE = int(Resolution)
channels = 1

def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    #img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.rgb_to_grayscale(img)
    if int(Resolution) > IMG_SIZE:
      img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

In [ ]:
print("Number of elements in the train dataset:", tf.data.experimental.cardinality(train_dataset).numpy())
print("Number of elements in the test dataset:", tf.data.experimental.cardinality(test_dataset).numpy())

Number of elements in the train dataset: 77871
Number of elements in the test dataset: 18354


In [ ]:
batch_size = 128

train_dataset = train_dataset.shuffle(len(labels_train))
train_dataset = train_dataset.map(preprocess_image)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(preprocess_image)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

### Model Config and Training

In [ ]:
# Modified Resnet model
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)    
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def ResNet34(shape = (IMG_SIZE, IMG_SIZE)):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation = 'relu')(x)
    # x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    x = tf.keras.layers.Dense(1)(x);
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    # model_plot(model);
    return model

In [ ]:
model = ResNet34();

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae','mape'])

In [ ]:
model.summary()
history = model.fit(train_dataset, epochs=20, validation_data=test_dataset)

Epoch 1/10
609/609 [==============================] - 887s 1s/step - loss: 78.0686 - mae: 4.2318 - mape: 14.3692 - val_loss: 13.8062 - val_mae: 2.9318 - val_mape: 11.7903
Epoch 2/10
609/609 [==============================] - 848s 1s/step - loss: 5.1733 - mae: 1.6596 - mape: 5.4815 - val_loss: 11.2904 - val_mae: 2.5235 - val_mape: 10.4598
Epoch 3/10
609/609 [==============================] - 848s 1s/step - loss: 2.6798 - mae: 1.1582 - mape: 3.8116 - val_loss: 31.4090 - val_mae: 4.1482 - val_mape: 16.1031
Epoch 4/10
609/609 [==============================] - 848s 1s/step - loss: 2.0348 - mae: 0.9925 - mape: 3.2850 - val_loss: 21.0462 - val_mae: 3.3367 - val_mape: 13.7935
Epoch 5/10
609/609 [==============================] - 848s 1s/step - loss: 1.9033 - mae: 0.9335 - mape: 3.0919 - val_loss: 59.6435 - val_mae: 5.8015 - val_mape: 23.0333
Epoch 6/10
609/609 [==============================] - 848s 1s/step - loss: 1.6659 - mae: 0.8604 - mape: 2.8591 - val_loss: 30.5458 - val_mae: 4.3886 - va

KeyboardInterrupt: ignored

In [ ]:
test_loss, test_mae, test_mape = model.evaluate(test_dataset)
print('Test loss:', test_loss)
print('Test mean absolute error:', test_mae)
print('Test mean absolute percentage error:', test_mape)

144/144 [==============================] - 44s 303ms/step - loss: 36.2253 - mae: 4.7119 - mape: 18.8093
Test loss: 36.22525405883789
Test mean absolute error: 4.711947441101074
Test mean absolute percentage error: 18.809282302856445
